# Import and install useful libraries

In [ ]:
!pip install praw -q
!pip install psaw -q

In [ ]:
import praw
import pandas as pd
from psaw import PushshiftAPI
import datetime 
from google.colab import files
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [ ]:
!mkdir CSV_FILES   # folder for saving the csv
%cd /content/CSV_FILES

mkdir: cannot create directory ‘CSV_FILES’: File exists
/content/CSV_FILES


# Authentication
Follow this article https://towardsdatascience.com/scraping-reddit-data-1c0af3040768 to get your client id, secret and agent details.

In [ ]:
# Authenticate
my_client_id = ''
my_client_secret = ''
my_user_agent = ''
api = PushshiftAPI()
reddit = praw.Reddit(client_id=my_client_id, client_secret=my_client_secret, user_agent=my_user_agent, check_for_async=False)

# Extract comments by URL

In [ ]:
# Function to scrape data from reddit and save it as a dataframe
posts_url = []

def reddit_scraper(urls):
  for url in urls:
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
      posts_url.append(comment.body)
  return posts_url

In [ ]:
urls = [
        'https://www.reddit.com/r/asklatinamerica/comments/n4omdw/colombia_protests_megathread_all_colombia_related/',
        'https://www.reddit.com/r/asklatinamerica/comments/n4fsua/what_the_fuck_is_going_on_in_colombia/',
        'https://www.reddit.com/r/asklatinamerica/comments/nef330/colombians_and_non_colombians_are_you_aware_of/'
        ]

df = pd.DataFrame(reddit_scraper(urls),columns=["body"])

In [ ]:
df

,body
0,Weekly discussion post will be restickied by tomorrow.\n\nEdit: please report comments that break rules. This thread has over 200 comments
1,How much the police brutality helped to the grow of the protest? People in favor of the government justified everything the police do?
2,Can someone explain the tax reform and what the governamment were proposing?
3,"Argentinians in this thread rn are like that ""you guys are getting paid?!"" guy from the meme"
4,"It's interesting to see that the proposed tax reform for Colombia is very similar to the one the Costa Rican government is trying to pass as part of its commitments to the IMF for a relief plan.\n\nCR is in a similar deficit situation and, thanks to the pandemic, the previous fiscal reform (from 2018 which saw VAT added to services and income tax hikes) showed how short sighted it was in not doing anything to curb spending. So they proposed several taxes last year including more income tax hikes and tripling property taxes. This resulted in a series of protests which made the government backpedal.\n\nThey learned from the experience and now introduced the new reform as part of the IMF commitments, with the most vague and surreptitious wording, trying to confuse people and avoid more protests. Things are astonishingly similar to Colombia: lowering the exempt ceiling and changing income thresholds that at least for me will result in a very hefty tax burden increase. It will be interesting to see what happens here when it enters actual discussion (the government does not have a majority and there are elections next year), so this unfortunate Colombia case can point a bit to what can happen here."
...,...
515,If you generate billions of dollars but it all pools at the top then you still haven't solved poverty. Reducing inequality is very effective at bringing people out of poverty.
516,New Zealand doesn't border the world's largest drug consumer who also sells weapons to the cartels. Reducing the government's size and power in countries like Mexico or Colombia would be insane for that same reason. Cartels would just step up and assume full political power.
517,"Latin America is not producing ""billions of dollars"", if we reduce ineaquality to the GINI point of 0 (every person receives the same income), everybody would still be as poor as the lower classes of Europe. Iraq is two times more equal than Chile."
518,"I'm saying if you were to create all that money, it is completely useless if it doesn't reach 99% of the population. Which it won't, because trickle-down economics are bullshit.\n\nAnd there are plenty of Latin American billionaires."


# Extract comments by keywords

In [ ]:
# Function to get the comments from subreddits results

start_time = int(datetime.datetime(2021,4,1).timestamp())
end_time = int(datetime.datetime(2021,8,27).timestamp())
posts = []

def get_comments(subreddit, keyword):
  # Get the URLs
  submissions = list(api.search_submissions(after= start_time, before=end_time,
                             subreddit = subreddit,
                             q = keyword,
                             filter = ['full_link', 'num_comments', ],
                             limit = 100))
  sub = pd.DataFrame(submissions)
  comments = sub[sub['num_comments']>0] #only post with comments
  comments.astype('str')
  urls = comments['full_link'].tolist()
# Get the comments
  for url in urls:
    submission = reddit.submission(url=url)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
      posts.append(comment.body)
  return posts


In [ ]:
# Creating dataframes for differents subreddits and keywords
df1 = pd.DataFrame(get_comments('Colombia', 'paro'),columns=["body"])    
df2 = pd.DataFrame(get_comments('Colombia', 'protesta'),columns=["body"])    
df3 = pd.DataFrame(get_comments('Colombia', 'strike'),columns=["body"])   
df4 = pd.DataFrame(get_comments('Colombia', 'ddhh'),columns=["body"])  

In [ ]:
# Data as a single dataframe
reddit_data = pd.concat([df, df1, df2, df3, df4], ignore_index=True )
reddit_data

,body
0,Weekly discussion post will be restickied by tomorrow.\n\nEdit: please report comments that break rules. This thread has over 200 comments
1,How much the police brutality helped to the grow of the protest? People in favor of the government justified everything the police do?
2,Can someone explain the tax reform and what the governamment were proposing?
3,"Argentinians in this thread rn are like that ""you guys are getting paid?!"" guy from the meme"
4,"It's interesting to see that the proposed tax reform for Colombia is very similar to the one the Costa Rican government is trying to pass as part of its commitments to the IMF for a relief plan.\n\nCR is in a similar deficit situation and, thanks to the pandemic, the previous fiscal reform (from 2018 which saw VAT added to services and income tax hikes) showed how short sighted it was in not doing anything to curb spending. So they proposed several taxes last year including more income tax hikes and tripling property taxes. This resulted in a series of protests which made the government backpedal.\n\nThey learned from the experience and now introduced the new reform as part of the IMF commitments, with the most vague and surreptitious wording, trying to confuse people and avoid more protests. Things are astonishingly similar to Colombia: lowering the exempt ceiling and changing income thresholds that at least for me will result in a very hefty tax burden increase. It will be interesting to see what happens here when it enters actual discussion (the government does not have a majority and there are elections next year), so this unfortunate Colombia case can point a bit to what can happen here."
...,...
6243,">ColombianLivesMatter\n\nNo, porfavor..."
6244,Totalmente de acuerdo con usted. De alguna manera la gente cree que rompiendo vidrios y robando tiendas el país va a progresar. Me encantaría saber cuál es la lógica detrás de eso.
6245,"Dicen que es necesario porque no los han escuchado antes, porque llevan años protestando y nada, pero en realidad es la primera vez que protestan por esto."
6246,"¿Cómo van a decir que llevan años protesta do por una reforma que se sacó hace unas semanas? Además, los actos vandálicos lo único que logran es desprestigiar la protesta, la gente en vez de apoyar le da la espalda a este tipo de movilizaciones precisamente cuando hay violencias."


In [ ]:
# Save as csv
reddit_data.to_csv('reddit_data.csv')